In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 39.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=598d99915ddcfbc8575a91c9dce808957fde822cd410538645e813d4c03f31f8
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.stat import Correlation
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .master("local[4]") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = spark.read.parquet('../../data/cleanedDataset_parquet')

In [3]:
from pyspark.ml.feature import QuantileDiscretizer 

qds = QuantileDiscretizer(relativeError=0.0001, handleInvalid="error", numBuckets=10, inputCol="popularity_track", outputCol="label")

df = qds.setHandleInvalid("keep").fit(df).transform(df)

In [5]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
assembler = VectorAssembler( 
inputCols=['age',
 'duration_ms',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'avg_artist_followers',
 'avg_artist_popularity',
 'sum_artist_followers',
 'sum_artist_popularity'], 
outputCol="features")
output=assembler.transform(df)

In [7]:
final_data = output.select( "features", "label")

In [8]:
from pyspark.ml.feature import StandardScaler


scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

# Normalize each feature to have unit standard deviation.
df = scalerModel.transform(final_data)

In [9]:
train, test = df.randomSplit([0.7, 0.3], seed = 10)

In [19]:
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

In [20]:
ovr = OneVsRest(classifier=lr)

In [21]:
ovrModel = ovr.fit(train)

In [22]:
predictions = ovrModel.transform(test)

In [23]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [24]:
accuracy = evaluator.evaluate(predictions)
#print("Test Error = %g" % (1.0 - accuracy))

Py4JJavaError: An error occurred while calling o5758.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 555.0 failed 1 times, most recent failure: Lost task 3.0 in stage 555.0 (TID 2484) (192.168.56.1 executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=3, Impossibile trovare il percorso specificato
	at java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:165)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apach

In [ ]:
accuracy